# Practice notebook for confidence intervals using NHANES data

[github link](https://github.com/karimkmafifi/Inferential-Statistical-Analysis-with-Python---Coursera/blob/master/nhanes_confidence_intervals_practice.ipynb)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import scipy.stats

da = pd.read_csv("nhanes_2015_2016.csv")

## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [2]:
da['DMDMARTL'] = da['DMDMARTL'].replace({1:'married', 2:'widowed', 
                                         3:'divorced', 4:'separated',
                                        5:'never married', 6:'living with partner',
                                        77:'refused', 99:"don't know", '.':'missing'})

da['DMDMARTL'].unique()

array(['married', 'divorced', 'living with partner', 'separated',
       'never married', nan, 'widowed', 'refused'], dtype=object)

In [3]:
da['RIAGENDR'] = da['RIAGENDR'].replace({1:'male', 2:'female', '.':'missing'})
da['RIAGENDR'].unique()

array(['male', 'female'], dtype=object)

In [4]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

In [5]:
# Get the women between 35 and 50. The value of RIAGENDR is equal to 2 if a subject is female
middle_age_women = da.query("RIDAGEYR >= 35 & RIDAGEYR <= 50 & RIAGENDR == 'female'")
middle_age_women.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
4,83736,2.0,1.0,1.0,2,female,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0
34,83799,NaN,NaN,NaN,2,female,37,2,1.0,4.0,...,110.0,72.0,66.6,161.6,25.5,NaN,NaN,NaN,NaN,2.0
50,83828,1.0,NaN,2.0,2,female,39,1,2.0,3.0,...,100.0,62.0,71.3,162.0,27.2,36.8,34.6,29.1,94.6,NaN
52,83832,2.0,1.0,4.0,2,female,50,1,2.0,1.0,...,NaN,NaN,105.9,157.7,42.6,29.2,35.0,40.7,129.1,NaN
55,83837,2.0,2.0,NaN,2,female,45,1,1.0,2.0,...,114.0,68.0,77.5,148.3,35.2,30.5,34.0,34.4,107.6,2.0


In [6]:
# Check that our data are correct! This step is not necessary but it is good practice :)
assert np.all(middle_age_women['RIAGENDR'] == 'female')
assert np.all(middle_age_women['RIDAGEYR'] <= 50)
assert np.all(middle_age_women['RIDAGEYR'] >= 35)

# assert လုပ်တာကမှန်/မမှန် စစ်တာ။ မမှန်ရင် AssertionError တက်မည်။

In [7]:
# Partition the group into married and non-married women
married_middle_age_women = middle_age_women.query("DMDMARTL == 'married'")
non_married_middle_age_women = middle_age_women.query("DMDMARTL != 'married'")

# More data checks!
assert np.all(married_middle_age_women['DMDMARTL'] == "married")
assert np.all(non_married_middle_age_women['DMDMARTL'] != "married")

In [18]:
# We now have checked our data is valid and is partitioned correctly. We 
# now can compute the 90% confidence intervals for the proportion which have
# completed college. This is coded in the DMDEDUC2 variable and it is equal to
# 5 if they have completed college or above
married_and_completed_college = married_middle_age_women['DMDEDUC2'] == 5
p_hat_married = np.mean(married_and_completed_college)
married_sample_size = married_and_completed_college.size
print(married_sample_size)
"The proportion of married middle age women (N={}) who completed college is: {:.2f}".\
format(married_sample_size, p_hat_married)

449


'The proportion of married middle age women (N=449) who completed college is: 0.36'

In [17]:
# We can do the same for non-married women
non_married_and_completed_college = non_married_middle_age_women['DMDEDUC2'] == 5
p_hat_non_married = np.mean(non_married_and_completed_college)
non_married_sample_size = non_married_and_completed_college.size
print(non_married_sample_size)
"The proportion of non-married middle age women (N={}) who completed college is: {:.2f}" \
.format(non_married_sample_size, p_hat_non_married)

338


'The proportion of non-married middle age women (N=338) who completed college is: 0.21'

In [31]:
# We can now compute the confidence intervals. Remember, for a two-sided 
# confidence interval, we need 5% in each of the tails and 95% PPF will give us
# this value !
z_multiplier = scipy.stats.norm.ppf(q = 0.95) # ppf(q, loc=0, scale=1)
print(z_multiplier)
married_standard_error = np.sqrt(p_hat_married * (1 - p_hat_married) / married_sample_size)
ci_married_lower_bound = p_hat_married - z_multiplier * married_standard_error
ci_married_upper_bound = p_hat_married + z_multiplier * married_standard_error
"A 90% confidence interval for the proportion of married women who completed college is ({:.2f}, {:.2f})".format(
    ci_married_lower_bound, 
    ci_married_upper_bound
)

# ppf = Percent point function (inverse of cdf — percentiles).

1.6448536269514722


'A 90% confidence interval for the proportion of married women who completed college is (0.32, 0.40)'

In [16]:
# We now can do the same for non-married women
z_multiplier = scipy.stats.norm.ppf(q = 0.95)
non_married_standard_error = np.sqrt(p_hat_non_married * (1 - p_hat_non_married) / non_married_sample_size)
ci_non_married_lower_bound = p_hat_non_married - z_multiplier * non_married_standard_error
ci_non_married_upper_bound = p_hat_non_married + z_multiplier * non_married_standard_error
"A 90% confidence interval for the proportion of non-married women who completed college is ({:.2f}, {:.2f})" \
.format(ci_non_married_lower_bound, ci_non_married_upper_bound)

'A 90% confidence interval for the proportion of non-married women who completed college is (0.18, 0.25)'

__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

The width of the confidence interval for the population proportion of married women who completed college is __eight percentage__ points wide while the confidence interval for non-married women is __seven points__ wide. The confidence interval for married women is larger, despite having a larger sample size, because the $\hat{p}_{married}$ estimate is closer to 50% than the $\hat{p}_{non-married}$ so the standard error of $\hat{p}_{married}$ is wider.

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

We can see that, on average, a middle age woman who is married is more likely to have completed college than a middle age women who is not married, and the confidence intervals for these two estimates do not overlap.

## Question 2

Construct a 95% confidence interval for the proportion of smokers who are female. Construct a 95% confidence interval for the proportion of smokers who are male. Construct a 95% confidence interval for the **difference** between those two gender proportions.

In [48]:
da["SMQ020x"]= da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "male", 2: "female"})

In [52]:
# First, lets prepare our data
females = da.query('RIAGENDRx == "female"')
males = da.query("RIAGENDRx == 'male'")

# I found the smoking variable here: https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/SMQ_I.htm#SMQ020
is_female_smoker = females['SMQ020x'] == 'Yes'
is_male_smoker = males['SMQ020x'] == 'Yes'

In [58]:
# In order to reduce the chance of mistakes, I'm going to create a simple function
# for computing confidence intervals. This function will do the exact same steps as 
# above so nothing should be scary :)
def MakeAConfidenceInterval(data, confidence):
    p_hat = np.mean(data)
    n = data.size
    z_multiplier = scipy.stats.norm.ppf(q = 1 - (1 - confidence) / 2)
    print(z_multiplier)
    standard_error = np.sqrt(p_hat * (1 - p_hat) / n)
    lower_bound = p_hat - z_multiplier * standard_error
    upper_bound = p_hat + z_multiplier * standard_error
    return {"phat": p_hat, "n":n, "lower_bound":lower_bound, "upper_bound":upper_bound}

In [59]:
female_smoking_ci = MakeAConfidenceInterval(data = is_female_smoker, confidence = 0.95)
"The CI for female smokers (N={}) is ({:.2f}, {:.2f})".format(
    female_smoking_ci['n'],
    female_smoking_ci['lower_bound'],
    female_smoking_ci['upper_bound']
)

1.959963984540054


'The CI for female smokers (N=2976) is (0.29, 0.32)'

In [60]:
male_smoking_ci = MakeAConfidenceInterval(data = is_male_smoker, confidence = 0.95)
"A CI for male smokers (N={}) is ({:.2f}, {:.2f})".format(
    male_smoking_ci['n'],
    male_smoking_ci['lower_bound'],
    male_smoking_ci['upper_bound']
)

1.959963984540054


'A CI for male smokers (N=2759) is (0.49, 0.53)'

### Two proportion formula

In [61]:
# Now we can construct the confidence interval for the difference between the proportions.
# This formula is pretty long so we will break it down into parts

# First compute the standard error of the difference
standard_error = np.sqrt(
    female_smoking_ci['phat'] * (1 - female_smoking_ci['phat']) / female_smoking_ci['n'] + 
    male_smoking_ci['phat'] * (1 - male_smoking_ci['phat']) / male_smoking_ci['n']
)

# Get the z-multiplier for our 95% confidence interval
z_multiplier = scipy.stats.norm.ppf(q = 0.975)

# Compute the point estimate for the differences between the male and female smoking proportion
difference = male_smoking_ci['phat'] - female_smoking_ci['phat']

# Compute the lower_bound and the upper_bound of the confidence interval
lower_bound = difference - z_multiplier * standard_error
upper_bound = difference + z_multiplier * standard_error
"A CI for the difference between male smoking proportion less the female smoking proportion is ({:.2f}, {:.2f})".format(
  lower_bound,
  upper_bound
)

'A CI for the difference between male smoking proportion less the female smoking proportion is (0.18, 0.23)'

__Q2a.__ Why might it be relevant to report the separate gender proportions **and** the difference between the gender proportions?

When we look at the confidence interval for the males who smoke and the proportion of females who smoke, we see how they behave marginally. That is, we see how they behave in their own group. But in order to compare them directly, we also have to compute the confidence interval for the difference. This makes sure that the two samples are compared fairly and we, as much as possible, control for sample size differences. 

__Q2b.__ How does the **width** of the confidence interval for the difference between the gender proportions compare to the widths of the confidence intervals for the separate gender proportions?

The width for the difference in proportions will always be larger than the widths of the two intervals it is made from. This happens because of the way the standard error is composed. In general, $\sqrt{A} < \sqrt{A + B}$ when $B > 0$

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters.

In [62]:
# First we get the data and look at it
height_data = da['BMXHT']
height_data.head()

0    184.5
1    171.4
2    170.1
3    160.9
4    164.9
Name: BMXHT, dtype: float64

__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial differences in the confidence interval widths that you see.

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?

__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

__Q6a.__ Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.  

__Q6b.__ Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?